In [1]:
import pynq
import torch
import torchvision
import time
import numpy as np
import matplotlib.pyplot as plt

from copy import deepcopy

print(f"Using PYNQ {pynq.__version__}")
!lscpu

/home/dpan/.conda/envs/pynq/lib/python3.9/site-packages/pydantic/_internal/_config.py:317: UserWarning: Valid config keys have changed in V2:
* 'underscore_attrs_are_private' has been removed
  warnings.warn(message, UserWarning)
/home/dpan/.conda/envs/pynq/lib/python3.9/site-packages/pydantic/_internal/_config.py:317: UserWarning: Valid config keys have changed in V2:
* 'underscore_attrs_are_private' has been removed
  warnings.warn(message, UserWarning)


Using PYNQ 3.0.1
Architecture:        x86_64
CPU op-mode(s):      32-bit, 64-bit
Byte Order:          Little Endian
CPU(s):              16
On-line CPU(s) list: 0-15
Thread(s) per core:  2
Core(s) per socket:  8
Socket(s):           1
NUMA node(s):        1
Vendor ID:           GenuineIntel
CPU family:          6
Model:               167
Model name:          11th Gen Intel(R) Core(TM) i7-11700K @ 3.60GHz
Stepping:            1
CPU MHz:             4883.768
CPU max MHz:         5000.0000
CPU min MHz:         800.0000
BogoMIPS:            7200.00
Virtualization:      VT-x
L1d cache:           48K
L1i cache:           32K
L2 cache:            512K
L3 cache:            16384K
NUMA node0 CPU(s):   0-15
Flags:               fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pge mca cmov pat pse36 clflush dts acpi mmx fxsr sse sse2 ss ht tm pbe syscall nx pdpe1gb rdtscp lm constant_tsc art arch_perfmon pebs bts rep_good nopl xtopology nonstop_tsc cpuid aperfmperf tsc_known_freq pni pclmulqdq dt

In [2]:
XCLBIN_PATH = "/home/dpan/project_240d/gemm_hls/build/MatrixMultiplication_hw.xclbin"

ol = pynq.Overlay(XCLBIN_PATH)
mm_kernel = ol.MatrixMultiplicationKernel_1


In [3]:
N = 512
K = 1024
M = 50176

A = pynq.allocate(shape=(N, K), dtype=np.float32)
B = pynq.allocate(shape=(K, M), dtype=np.float32)
C = pynq.allocate(shape=(N, M), dtype=np.float32)

A[:] = np.random.rand(N, K)
B[:] = np.random.rand(K, M)

print(A.shape, B.shape, C.shape)

A.sync_to_device()
B.sync_to_device()
C.sync_to_device()

start = time.time()
mm_kernel.call(A, B, C, N, K, M)
elapsed = time.time() - start

C.sync_from_device()

cpu_start = time.time()
truth = A @ B
cpu_elapsed = time.time() - cpu_start

print(A.shape, B.shape, C.shape)

err = np.sum(truth - C)
print(f"Equal: {np.isclose(truth, C).all()}, err={err}")
print(f"Kernel took {elapsed:.4f}s, throughput: {N * K * M / (1 << 30) / elapsed} GFLOPS")
print(f"CPU reference time {cpu_elapsed:.4f}s")

(512, 1024) (1024, 50176) (512, 50176)
(512, 1024) (1024, 50176) (512, 50176)
Equal: True, err=2.2233123779296875
Kernel took 0.7188s, throughput: 34.08467264461544 GFLOPS
CPU reference time 0.1481s


In [ ]:
MEM_GRANULARITY = 512

N, K = _A.shape
K, M = _B.shape

Nd = N if N // MEM_GRANULARITY == 0 else N // MEM_GRANULARITY + MEM_GRANULARITY
Kd = K if K // MEM_GRANULARITY == 0 else K // MEM_GRANULARITY + MEM_GRANULARITY
Md = M if M // MEM_GRANULARITY == 0 else M // MEM_GRANULARITY + MEM_GRANULARITY

print(Nd, Kd, Md)

A = pynq.allocate(shape=(N, K), dtype=np.float32)
B = pynq.allocate(shape=(K, M), dtype=np.float32)
C = pynq.allocate(shape=(N, M), dtype=np.float32)

A[:] = _A[:]
B[:] = _B[:]

print(A.shape, B.shape, C.shape)

A.sync_to_device()
B.sync_to_device()
C.sync_to_device()

NameError: name '_A' is not defined

In [3]:
ol.free()

NameError: name 'ol' is not defined